In [1]:
import pandas as pd

In [6]:
#全局配置
FOLDER='/Users/includeno/Documents/GitHub/includeno/news-community-back-end/recommender_pyspark'
DATASET_FOLDER='data'


#movie_lens
ML_100k_CSV=DATASET_FOLDER+"/ml-100k/u.data"
ML_100k_delimiter="\t"

ML_1m_CSV=DATASET_FOLDER+"/ml-1m/ratings.dat"
ML_1m_delimiter="::"
ML_1m_MOVIES_CSV=DATASET_FOLDER+"/movies.dat"

def ml_rating(size):
    if(size=='1m'):
        return ML_1m_CSV,ML_1m_delimiter
    if(size=='100k'):
        return ML_100k_CSV,ML_100k_delimiter
    

In [7]:
movies_title = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv(ML_1m_MOVIES_CSV, sep=ML_1m_delimiter, header=None, names=movies_title,engine='python', encoding='ISO-8859-1')
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies.Title.values

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Tigerland (2000)', 'Two Family House (2000)',
       'Contender, The (2000)'], dtype=object)

In [10]:
titles=pd.DataFrame(movies)
titles.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
len(titles.Title.values)

3883

In [12]:
infos_columns=['id', 'name', 'url', 'time', 'genre', 'release_time', 'intro',
       'directors', 'writers', 'starts', 'completed', 'create_time',
       'update_time', 'full_name']

infos_file=DATASET_FOLDER+'/merged.csv'
infos=pd.read_csv(infos_file)
infos.head()


,id,name,url,time,genre,release_time,intro,directors,writers,starts
0,1,Toy Story (1995),https://m.media-amazon.com/images/M/MV5BMDU2ZW...,1h 21min,Animation|Adventure|Comedy,22 November 1995 (USA),A cowboy doll is profoundly threatened and jea...,John Lasseter,John Lasseter|Pete Docter,Tom Hanks|Tim Allen|Don Rickles
1,2,Jumanji (1995),https://m.media-amazon.com/images/M/MV5BZTk2Zm...,1h 44min,Adventure|Comedy|Family,15 December 1995 (USA),When two kids find and play a magical board ga...,Joe Johnston,Jonathan Hensleigh|Greg Taylor,Robin Williams|Kirsten Dunst|Bonnie Hunt
2,3,Grumpier Old Men (1995),https://m.media-amazon.com/images/M/MV5BMjQxM2...,1h 41min,Comedy|Romance,22 December 1995 (USA),John and Max resolve to save their beloved bai...,Howard Deutch,Mark Steven Johnson,Walter Matthau|Jack Lemmon|Ann-Margret
3,4,Waiting to Exhale (1995),https://m.media-amazon.com/images/M/MV5BYzcyMD...,2h 4min,Comedy|Drama|Romance,22 December 1995 (USA),"Based on Terry McMillan's novel, this film fol...",Forest Whitaker,Terry McMillan|Terry McMillan,Whitney Houston|Angela Bassett|Loretta Devine
4,5,Father of the Bride Part II (1995),https://m.media-amazon.com/images/M/MV5BOTEyNz...,1h 46min,Comedy|Family|Romance,8 December 1995 (USA),George Banks must deal not only with the pregn...,Charles Shyer,Albert Hackett|Frances Goodrich,Steve Martin|Diane Keaton|Martin Short


In [13]:
infos_title=pd.DataFrame(infos['name'])
infos_title.head()

,name
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)


In [14]:
#infos_title.rename({'name':'Title'}, axis=1, inplace=True)
infos_title.head()

,name
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)


In [15]:
left=titles.Title.values
left

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Tigerland (2000)', 'Two Family House (2000)',
       'Contender, The (2000)'], dtype=object)

In [16]:
right=infos_title.name.values
right

array(['Toy Story (1995) ', 'Jumanji (1995) ', 'Grumpier Old Men (1995) ',
       ..., 'Flint (2017) ', 'Bungou Stray Dogs: Dead Apple (2018) ',
       'Dice Rules (1991) '], dtype=object)

In [17]:
import nltk
from nltk.tokenize import word_tokenize

# tokens = word_tokenize('Toy Story (1995)')
# sor=sorted(tokens)
# sor

left_2=[]
for item in left:
    tokens = word_tokenize(str(item))
    sor=sorted(tokens)
    tk=[]
    #print(sor)
    for sorted_item in sor:
        #print(sorted_item)
        if(str(sorted_item) in ['(',')',',',"'"]):
            continue
        # if(str(sorted_item).startswith('19') or str(sorted_item).startswith('20')):
        #     continue
        tk.append(str(sorted_item))
    left_2.append(' '.join(tk))
    #print(tk)

right_2=[]
for item in right:
    tokens = word_tokenize(str(item))
    sor=sorted(tokens)
    tk=[]
    
    for sorted_item in sor:
        #print(sorted_item)
        if(str(sorted_item) in ['(',')',',',"'"]):
            continue
        # if(str(sorted_item).startswith('19') or str(sorted_item).startswith('20')):
        #     continue
        tk.append(str(sorted_item))
    right_2.append(' '.join(tk))

names_in_movies=set(right_2)

result=[]
for item in left_2:
    if(item not in names_in_movies):
        #print(item)
        result.append(item)
    else:
        names_in_movies.remove(item)

print(len(result))
print(result)

1404
['1995 Persuasion', '1995 Children City Lost The of', '1995 Shanghai Triad Yao a dao qiao waipo yao yao', '1995 Courage Wings of', '1995 Carrington', '1995 Across Sea Time of the', '1995 Se7en Seven', '1994 Angel Guardian', '1995 Kids Round Table of the', '1994 Il Postino Postman The', '1995 Confessional Confessionnal Le The', '1996 Sea Two by if', '1995 Bits Two', '1995 French Gazon Twist maudit', '1995 Les Misérables', '1995 Badkonake Balloon Sefid The White', "'re 1995 Dead Denver Do Things You in to when", "'s 1995 Antonia Antonia Line", '1995 Hamptons Last Summer in the', '1995 August Journey King The of', '1995 Haine Hate La', '1994 Shopping', '1994 Keiner Loves Me Nobody liebt mich', '1995 Bar Body Headless Topless in', '1995 Bronx Rumble in the', "'s 1995 Margaret Museum", '1995 Field Happiness Is in the', '1996 Race Sun the', '1993 Boys St. The Vincent of', '1994 Chungking Express', '1995 L Maker Star The Uomo delle stelle', '1996 Disaster Flirting With', '1994 III NeverE

In [24]:
import nltk
from nltk.tokenize import word_tokenize
def get_tokens(item):
    tokens = word_tokenize(str(item))
    sor=sorted(tokens)
    tk=[]
    for sorted_item in sor:
        #print(sorted_item)
        if(str(sorted_item) in ['(',')',',',"'",'The','the']):
            continue
        # if(str(sorted_item).startswith('19') or str(sorted_item).startswith('20')):
        #     continue
        tk.append(str(sorted_item))
    return ' '.join(tk)



In [25]:
titles['new_title']=titles['Title'].apply(get_tokens)
titles.head()

,MovieID,Title,Genres,new_title
0,1,Toy Story (1995),Animation|Children's|Comedy,1995 Story Toy
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995 Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995 Grumpier Men Old
3,4,Waiting to Exhale (1995),Comedy|Drama,1995 Exhale Waiting to
4,5,Father of the Bride Part II (1995),Comedy,1995 Bride Father II Part of


In [26]:
infos_title['new_title']=infos_title['name'].apply(get_tokens)
infos_title.head()

,name,new_title
0,Toy Story (1995),1995 Story Toy
1,Jumanji (1995),1995 Jumanji
2,Grumpier Old Men (1995),1995 Grumpier Men Old
3,Waiting to Exhale (1995),1995 Exhale Waiting to
4,Father of the Bride Part II (1995),1995 Bride Father II Part of


In [27]:
collects=[]
for i, row in titles.iterrows():
    if(row['new_title'] not in infos_title.new_title.values):
        print(row['Title'],"========",row['new_title'])
        collects.append(row)
print(len(collects))



Persuasion (1995) ======== 1995 Persuasion
City of Lost Children, The (1995) ======== 1995 Children City Lost of
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) ======== 1995 Shanghai Triad Yao a dao qiao waipo yao yao
Wings of Courage (1995) ======== 1995 Courage Wings of
Carrington (1995) ======== 1995 Carrington
Across the Sea of Time (1995) ======== 1995 Across Sea Time of
Seven (Se7en) (1995) ======== 1995 Se7en Seven
Guardian Angel (1994) ======== 1994 Angel Guardian
Kids of the Round Table (1995) ======== 1995 Kids Round Table of
Postino, Il (The Postman) (1994) ======== 1994 Il Postino Postman
Confessional, The (Le Confessionnal) (1995) ======== 1995 Confessional Confessionnal Le
Two if by Sea (1996) ======== 1996 Sea Two by if
Two Bits (1995) ======== 1995 Bits Two
French Twist (Gazon maudit) (1995) ======== 1995 French Gazon Twist maudit
Misérables, Les (1995) ======== 1995 Les Misérables
White Balloon, The (Badkonake Sefid ) (1995) ======== 1995 Badkonake Balloon Sefid 

In [22]:
data=pd.DataFrame(collects)
data.head()

data.to_csv('data/diff.csv',index=False)